After a super-long break from writing blogs, I've decided to get back to it this year.

I've always been interested in systems programming, but somehow never _really_ picked it up. The rate of progress in the GenAI space has been exponential recently, with players like Google [@Google] reportedly processing 9.7 trillion tokens a month. Companies are now investing more time and resources interest in making these Large Language Modelsas fast and cheap as possible, by improving training and inference efficiency using "moar" compute.

I briefly spoke about [GPU computing last year](https://www.figma.com/deck/Sq9frEEoTFgFWthOJ4EM5w/intro_gpu_cuda?node-id=1-37&t=VNzh9p2qKrHNSTJj-1), and finally decided to learn it this summer. The goal is to eventually be able to implement kernels for fast matmuls, softmax, and FlashAttention.

## Why Mojo?

I've tried learning Rust [multiple](https://github.com/goodhamgupta/rustlings) [times](https://github.com/goodhamgupta/100-exercises-to-learn-rust/), along with a few stints of trying C, C++ and Zig, but I never really felt as comfortable in these languages as I did in Python.

In early 2023, Modular announced Mojo🔥, a new systems-programming language promising:

- Python-like syntax
- Support for both CPU and GPU architectures
- Kernel autofusion
- Builds on MLIR
- Traits and bounds checking
- Interopeability with PTX

Modular has since announced Max, their AI inference platform, built on Mojo. The released [all kernels](https://github.com/modular/modular/tree/main/max/kernels) available as part of the platform, along with their own version[@modularpuzzles] of Sasha Rush's GPU Puzzles [@GPUPuzzles] in Mojo. IMO, their kernels were much easier to read compared to CUDA/Triton implementations, so I decided to learn a bit more about how to write these kernels.

## GPUs 101

Not sure what to put in here. Skipping for now.

## Infrastructure

If you plan on solving these puzzles, remember to pick a [compatible GPU](https://docs.modular.com/max/faq/#gpu-requirements) and follow the [setup instructions](https://builds.modular.com/puzzles/howto.html)

I completed the puzzles on a instance with a RTX4090 Ti chip, rented via [Prime Intellect](https://www.primeintellect.ai/) at **0.22 $/hr**!

## Part 1: GPU Fundamentals

The Modular team has created beautiful [Manim](https://github.com/ManimCommunity/manim) visualizations for each puzzle, making the concepts much more intuitive. I'll walk through these visualizations as we tackle each problem.

### Puzzle 1: Map

In this puzzle, we aim to add a scalar to a vector. Specifically, we want to use a separate thread for each element in the vector, add the scalar, and write the result to the output memory.

When we create the kernel, the scalar will be effectively "broadcast" or expanded to match the shape of the input vector. This allows each element of the vector to be independently added with the scalar value in parallel by its dedicated thread, following the [broadcasting rules](https://docs.pytorch.org/docs/stable/notes/broadcasting.html).

![](mojo_gpu_puzzles/p01_vector_addition.png){fig-align="middle"}

<details open>
<summary> Solution </summary>
```{.mojo filename="p01.mojo"}
fn add_10(out: UnsafePointer[Scalar[dtype]], a: UnsafePointer[Scalar[dtype]]):
    i = thread_idx.x
    # FILL ME IN (roughly 1 line)
    out[i] = a[i] + 10
```

```bash
pixi run p01
# out: HostBuffer([10.0, 11.0, 12.0, 13.0])
# expected: HostBuffer([10.0, 11.0, 12.0, 13.0])
```

</details>


### Puzzle 2: Zip

This is an extension of the map puzzle. Now, we aim to add 2 tensors together.

![](mojo_gpu_puzzles/p02.png)

As in puzzle 1, the aim is to use one individual thread for elements at a specific index in both vectors.

![](https://builds.modular.com/puzzles/puzzle_02/media/videos/720p30/puzzle_02_viz.gif)


<details open>
<summary> Solution </summary>
```{.mojo filename="p02.mojo"}
fn add_10(out: UnsafePointer[Scalar[dtype]], a: UnsafePointer[Scalar[dtype]]):
    i = thread_idx.x
    # FILL ME IN (roughly 1 line)
    out[i] = a[i] + 10
```

```bash
pixi run p01
# out: HostBuffer([10.0, 11.0, 12.0, 13.0])
# expected: HostBuffer([10.0, 11.0, 12.0, 13.0])
```

</details>